In [1]:
import os
os.sys.path.append("C:\\Users\\markp\\repos\\PlittGiocomo_CA1Morph_2019")
import numpy as np
import scipy as sp
from scipy import signal
import sklearn as sk
from sklearn import neighbors
import preprocessing as pp
import utilities as u
import MAPInference as mapi
from matplotlib import pyplot as plt
import pickle



%matplotlib inline

%load_ext autoreload
%autoreload 2

In [9]:
df = pp.load_session_db()
df = df[df['RewardCount']>20]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
df = df[df["Track"]=="TwoTower_foraging"]
df = df[(df['ImagingRegion']=='CA1' )|(df['ImagingRegion']=='')]
df = df[df["Imaging"]==1]

In [17]:
df = pp.load_session_db()
df = df[df['RewardCount']>20]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
tracks="TwoTower_foraging|TwoTower_noTimeout"
df = df[df['Track'].str.contains(tracks,regex=True)]
df = df[(df['ImagingRegion']=='CA1' )|(df['ImagingRegion']=='')]
df = df[df["Imaging"]==1]

In [18]:
df

,MouseName,DateFolder,SessionNumber,Track,RewardCount,Imaging,ImagingRegion,DateTime,data file,scanmat,s2pfolder
413,4139202.2,20_07_2018,1,TwoTower_noTimeout,22,1,CA1,2018-07-20,G:\My Drive\VR_Data\TwoTower\4139202.2\20_07_2...,G:\My Drive\2P_Data\TwoTower\4139202.2\20_07_2...,None
414,4139202.2,20_07_2018,2,TwoTower_noTimeout,21,1,CA1,2018-07-20,G:\My Drive\VR_Data\TwoTower\4139202.2\20_07_2...,G:\My Drive\2P_Data\TwoTower\4139202.2\20_07_2...,None
423,4139202.2,22_07_2018,2,TwoTower_noTimeout,37,1,CA1,2018-07-22,G:\My Drive\VR_Data\TwoTower\4139202.2\22_07_2...,G:\My Drive\2P_Data\TwoTower\4139202.2\22_07_2...,None
606,4139212.2,08_10_2018,5,TwoTower_noTimeout,64,1,CA1,2018-10-08,G:\My Drive\VR_Data\TwoTower\4139212.2\08_10_2...,G:\My Drive\2P_Data\TwoTower\4139212.2\08_10_2...,None
608,4139212.2,08_10_2018,7,TwoTower_noTimeout,29,1,CA1,2018-10-08,G:\My Drive\VR_Data\TwoTower\4139212.2\08_10_2...,G:\My Drive\2P_Data\TwoTower\4139212.2\08_10_2...,None
...,...,...,...,...,...,...,...,...,...,...,...
3621,4343711.1,17_03_2020,1,TwoTower_foraging,41,1,CA1,2020-03-17,G:\My Drive\VR_Data\TwoTower\4343711.1\17_03_2...,G:\My Drive\2P_Data\TwoTower\4343711.1\17_03_2...,None
3605,4343717.2,16_03_2020,1,TwoTower_foraging,68,1,CA1,2020-03-16,G:\My Drive\VR_Data\TwoTower\4343717.2\16_03_2...,None,None
3639,4343717.2,18_03_2020,3,TwoTower_foraging,55,1,CA1,2020-03-18,G:\My Drive\VR_Data\TwoTower\4343717.2\18_03_2...,None,None
3673,4343717.2,20_03_2020,2,TwoTower_foraging,53,1,CA1,2020-03-20,G:\My Drive\VR_Data\TwoTower\4343717.2\20_03_2...,None,None


In [3]:
df = pp.load_session_db()
df = df[df['RewardCount']>20]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
df = df[df["Track"]=="TwoTower_foraging"]
df = df[(df['ImagingRegion']=='DG' )|(df['ImagingRegion']=='')]
df = df[df["Imaging"]==1]

In [3]:
df = pp.load_session_db()
df = df[df['RewardCount']>30]
df = df[df['Imaging']==1]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])

tracks = 'FreqMorph_Decision|FreqMorph_Timeout|TwoTower_noTimeout|TwoTower_Timeout'
df = df[df['Track'].str.contains(tracks,regex=True)]
fd_mice = '4222157.4|4222169.1|4222169.2|4222169.4'
df = df[df['MouseName'].str.contains(fd_mice,regex=True)]



In [23]:
# mouse = '4222168.1'
# mice = ['4343706']
mice = ['4343717.2'] #'4222169.1','4222169.2','4222169.4']
for mouse in mice:
    df_mouse = df[df["MouseName"]==mouse]

    try:
        os.makedirs(os.path.join("D:\\Suite2P_Data",mouse))
    except:
        pass

    for i in [-1]: #range(df_mouse.shape[0]):
        sess = df_mouse.iloc[i]
#         sess['scanmat']=pp.build_2P_filename(sess,serverDir = "G:\\My Drive\\2P_Data\\TwoTower")
#         sess['s2pfolder']=pp.build_s2p_folder(sess,serverDir = "G:\\My Drive\\2P_Data\\TwoTower")
        sess['scanmat']=pp.build_2P_filename(sess,serverDir = "G:\\My Drive\\2P_Data\\Syt3KO")
        sess['s2pfolder']=pp.build_s2p_folder(sess,serverDir = "G:\\My Drive\\2P_Data\\Syt3KO")
        fname = os.path.join("D:\\Suite2P_Data\\",mouse,"%s_%s_%i.pkl" % (sess['Track'],sess["DateFolder"],sess['SessionNumber']))
        if not os.path.exists(fname):           
            try:
                VRDat, C, S, dFF = pp.load_scan_sess(sess,fneu_coeff=.7)
                trial_info, tstart_inds, teleport_inds = u.by_trial_info(VRDat)
                S_trial_mat, occ_trial_mat, edges,centers = u.make_pos_bin_trial_matrices(S,
                                                            VRDat['pos']._values,tstart_inds,
                                                            teleport_inds,bin_size=10,
                                                            speed = VRDat['speed']._values)

                with open(os.path.join("D:\\Suite2P_Data\\",mouse,"%s_%s_%i.pkl" % (sess['Track'],sess["DateFolder"],sess['SessionNumber'])),"wb") as f:
                    pickle.dump({'VRDat':VRDat,'C':C,'S':S,'dFF':dFF,'trial_info':trial_info,'tstart_inds':tstart_inds,'teleport_inds':teleport_inds,
                                'S_trial_mat':S_trial_mat,'occ_trial_mat':occ_trial_mat,'edges':edges,'centers':centers},f)
            except:
                print(sess)

C:\Users\markp\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\markp\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


num aberrant ttls 0.0
8.668393127842345 3042.0210965133906
8.73168266801415 3041.9242041435073
frame diff -1


In [ ]:
with open(os.path.join("D:\\Suite2P_Data\\",'4343706','TwoTower_forag.p'),"rb") as f:
    data = pickle.load(f)
for k,v in data.items():
    print(k,v)

In [22]:
folders = os.walk(os.path.join("E:\\","2P_Data","TwoTower",mouse))
print(next(folders))

for 

('E:\\2P_Data\\TwoTower\\4139265.4', ['07_02_2019', '10_02_2019', '12_02_2019', '15_02_2019', '17_02_2019', '18_02_2019', '19_02_2019', '20_02_2019', '21_02_2019', '22_02_2019', 'test'], ['desktop.ini'])


In [19]:
for fold in folders:
    print(fold)

('E:\\2P_Data\\TwoTower\\4139265.4', ['07_02_2019', '10_02_2019', '12_02_2019', '15_02_2019', '17_02_2019', '18_02_2019', '19_02_2019', '20_02_2019', '21_02_2019', '22_02_2019', 'test'], ['desktop.ini'])
('E:\\2P_Data\\TwoTower\\4139265.4\\07_02_2019', ['TwoTower_foraging'], ['desktop.ini'])
('E:\\2P_Data\\TwoTower\\4139265.4\\07_02_2019\\TwoTower_foraging', ['TwoTower_foraging_002_001', 'TwoTower_foraging_004_004'], ['desktop.ini', 'TwoTower_foraging_002_001.mat', 'TwoTower_foraging_002_001.sbx', 'TwoTower_foraging_002_002.mat', 'TwoTower_foraging_002_002.sbx', 'TwoTower_foraging_002_003.sbx', 'TwoTower_foraging_004_004.mat', 'TwoTower_foraging_004_004.sbx'])
('E:\\2P_Data\\TwoTower\\4139265.4\\07_02_2019\\TwoTower_foraging\\TwoTower_foraging_002_001', ['suite2p'], ['desktop.ini'])
('E:\\2P_Data\\TwoTower\\4139265.4\\07_02_2019\\TwoTower_foraging\\TwoTower_foraging_002_001\\suite2p', ['plane0'], ['data.bin', 'desktop.ini', 'ops1.npy'])
('E:\\2P_Data\\TwoTower\\4139265.4\\07_02_2019\\T